In this notebook, we want to increase the complexity of the transfer learning for the model SAM from meta. Inthe previous notebook, we tested a lean version of transfer learning with SAM. 

* 100 epochs
* 10 patience
* Pytorch only instead of mixing HuggingFace with PyTorch
    * It seems like it's hard to rely only on PyTorch

In [1]:
# Setting up the enviroment

%pip install -q transformers
%pip install -q datasets
%pip install -q Pillow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

cwd = os.getcwd()
print(cwd)

/home/mark/work/MultiModal


In [ ]:
from datasets import load_dataset, Image
data_dir = os.path.join(cwd, 'data')

one_class_flipped_dataset = "sentinel-1-flipped-one-class"
two_class_augmented_dataset = "sentinel-1-augmented-two-class"

current_data_dir = os.path.join(data_dir, one_class_flipped_dataset)
print("Current Data Directory"+current_data_dir)
current_data_dir 

/home/mark/work/MultiModal/data
Current Data Directory/home/mark/work/MultiModal/data/sentinel-1-flipped-one-class
